In [1]:
#Se hará un modelo capaz de reconocer conductas inapropiadas 
"""_summary_
        Implemtaciones posibles:
            Tooling: 
                Chatbot? Míneria de textos, de alguna red social, gmail, sistema de control?
            Institucional: 
                Sería útil para las empresas en detectar conductas inapropiadas tales como racismo,
                machismo, acoso sexual, por medios oficiales de comunicación.
                    datasets = machismo, acoso sexual, racismo, xenofobia
                Implementado como un sistema de control parental, para controlar textos peligrosos
                    datasets? 
    """

'_summary_\n        Implemtaciones posibles:\n            Tooling: \n                Chatbot? Míneria de textos, de alguna red social, gmail, sistema de control?\n            Institucional: \n                Sería útil para las empresas en detectar conductas inapropiadas tales como racismo,\n                machismo, acoso sexual, por medios oficiales de comunicación.\n                    datasets = machismo, acoso sexual, racismo, xenofobia\n                Implementado como un sistema de control parental, para controlar textos peligrosos\n                    datasets? \n    '

In [2]:
# Importes
import kagglehub
import os
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import nltk
import json
import random
import numpy as np
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
nltk.download('punkt_tab')


/home/jordi/entornos/entorno_sic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 05:09:12.399330: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 05:09:12.413970: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 05:09:12.510998: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 05:09:12.592821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to regi

True

In [3]:
#Acá irán los datasets, ahorita en concepto de uso


"""
    Es un dataset que nos permite detectar textos racistas, en base al target:
    target inicial {1:is racist, 0:non-racist}
"""
path = kagglehub.dataset_download("rishabhjohri/racismdetectiondataset")
print("Path to dataset files:", path)

Path to dataset files: /home/jordi/.cache/kagglehub/datasets/rishabhjohri/racismdetectiondataset/versions/1


In [4]:
df_racism = pd.read_csv('../data_sets/behaviors_data/RacismDetectionDataSet.csv')
rows , columns = df_racism.shape
print(f'Filas: {rows}\nColumnas: {columns}')
df_racism.head()

Filas: 1999
Columnas: 2


,Comment,Label
0,i was born a racist and I will die a racist I ...,1
1,bitch nigga miss me with that,1
2,if you aint bout that murder game pussy nigga ...,1
3,gay niggas couldnt wait to act like bitches to...,1
4,why deos a gorilla always have a frown because...,1


In [5]:
# Modificamos las etiquetas, y las columnas
# text -- emotion

df_racism = df_racism.rename(columns={'Comment':'text', 'Label':'emotion'})
df_racism['emotion'] = df_racism['emotion'].replace({1:'racist'})
df_racism = df_racism[df_racism['emotion'] != 0]
 #Quitaremos los 0, para el modelo ya entrenado en neutral?
 

In [6]:
df_racism['emotion'].unique()

array(['racist'], dtype=object)

In [7]:
"""_summary_
    Este dataset nos permite identificar bullyng en comentarios
    """

path = kagglehub.dataset_download("saifulislam7/cyberbullying-and-harassment-detection-using-ml")

print("Path to dataset files:", path)
df_discg = pd.read_csv('../data_sets/behaviors_data/discrimination_variety.csv')
df_discg

Path to dataset files: /home/jordi/.cache/kagglehub/datasets/saifulislam7/cyberbullying-and-harassment-detection-using-ml/versions/1


,Text,Label,Types
0,Ten outside soon doctor shake everyone treatme...,Not-Bullying,NaN
1,my life has come to a standstill and at this p...,Not-Bullying,NaN
2,girl this nigga make me sick to my stomach,Bullying,Ethnicity
3,I wanna fuck you,Bullying,Sexual
4,"Oh hey, you should be ashamed of your disgusti...",Not - Bullying,NaN
...,...,...,...
8447,Please help this Christian nationalist battle ...,Bullying,Vocational
8448,Has the Holy Quran ever been read by the membe...,Bullying,Sexual
8449,And that is the difficulty - you need basic gu...,Bullying,Religion
8450,"As they will only encounter Muslim women, help...",Bullying,Religion


In [8]:
rows, colums = df_discg.shape
print(f'Filas: {rows}\nColumnas: {columns}')
df_discg['Label'] = df_discg['Label'].replace({'Not - Bullying':'Not-Bullyng','Not -Bullying':'Not-Bullying','Not- Bullying':'Not-Bullying',
                                              'Not-Bulying':'Not-Bullying','Not-Bullyng':'Not-Bullying','Not-Bulllying':'Not-Bullying',
                                              'Not- Bullying ':'Not-Bullying','Not-Bullying ':'Not-Bullying','Not -Bullying ':'Not-Bullying',
                                              'Not-Bullyng':'Not-Bullying','Not-Bulyying':'Not-Bullying',
                                              'Bulling ':'Bullying','Bullying.':'Bullying','Bullyibg':'Bullying','Bullying  ':'Bullying',
                                              'Bullyiing':'Bullying','Bullying ':'Bullying','Bullyiing':'Bullying','Bullyinfg':'Bullying'})
df_discg.loc[df_discg['Label'] == 'Not-Bullying', 'Types'] = 'Neutral'

df_discg = df_discg.dropna()
#print(df_discg['Types'].unique())
print(df_discg['Label'].unique())

Filas: 8452
Columnas: 2
['Not-Bullying' 'Bullying']


In [9]:
#Limpieza -> uniformidad de datos
df_discg = df_discg.rename(columns={'Text':'text','Types':'emotion'})
df_discg = df_discg.drop('Label', axis=1)
df_discg


,text,emotion
0,Ten outside soon doctor shake everyone treatme...,Neutral
1,my life has come to a standstill and at this p...,Neutral
2,girl this nigga make me sick to my stomach,Ethnicity
3,I wanna fuck you,Sexual
5,Haha isn't route running part of the position?...,Ethnicity
...,...,...
8447,Please help this Christian nationalist battle ...,Vocational
8448,Has the Holy Quran ever been read by the membe...,Sexual
8449,And that is the difficulty - you need basic gu...,Religion
8450,"As they will only encounter Muslim women, help...",Religion


In [10]:
"""_summary_
    Acoso sexual, etiquetas explícitas
    """

path = kagglehub.dataset_download("aadyasingh55/sexism-detection-in-english-texts")
print("Path to dataset files:", path)

Path to dataset files: /home/jordi/.cache/kagglehub/datasets/aadyasingh55/sexism-detection-in-english-texts/versions/1


In [11]:
df_sexh1 = pd.read_csv('../data_sets/behaviors_data/sexual_harassment1.csv')
df_sexh2 = pd.read_csv('../data_sets/behaviors_data/sexual_harassment2.csv')
df_sexh3 = pd.read_csv('../data_sets/behaviors_data/sexual_harassment3.csv')
df_sexh1

df_sexh = pd.concat([df_sexh1,df_sexh2,df_sexh3], ignore_index=True)
df_sexh

,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev
1,sexism2022_english-966,I bet she wished she had a gun,not sexist,none,none,dev
2,sexism2022_english-3553,I agree with that but at the same time I know ...,sexist,2. derogation,2.1 descriptive attacks,dev
3,sexism2022_english-13328,democrats from Minnesota (leftist muzzie shith...,sexist,2. derogation,2.2 aggressive and emotive attacks,dev
4,sexism2022_english-81,Texas jury sentences Jordanian immigrant to de...,not sexist,none,none,dev
...,...,...,...,...,...,...
19995,sexism2022_english-12034,I blew a shot at a record like that. Girl on t...,not sexist,none,none,train
19996,sexism2022_english-5228,girls really get fucked almost every weekend ?,not sexist,none,none,train
19997,sexism2022_english-10140,The hatred for moslems are Reasonable and Just...,not sexist,none,none,train
19998,sexism2022_english-9726,Now this is a woman who gets it. 👆,not sexist,none,none,train


In [12]:
df_sexh = df_sexh.drop('rewire_id',axis=1)
df_sexh = df_sexh.drop('label_category',axis=1)
df_sexh = df_sexh.drop('label_vector',axis=1)
df_sexh = df_sexh.drop('split',axis=1)
df_sexh = df_sexh[df_sexh['label_sexist'] != 'not sexist']

df_sexh

,text,label_sexist
2,I agree with that but at the same time I know ...,sexist
3,democrats from Minnesota (leftist muzzie shith...,sexist
7,aim for the head so she dosn't come back,sexist
9,This guys gonna have a great time in the barra...,sexist
13,Or women can stop being lying whores.,sexist
...,...,...
19968,The drama queen is just another open borders c...,sexist
19985,This is easily the dumbest thing ever written....,sexist
19986,Recently some thots started advertizing their ...,sexist
19988,I wish yall understand what a burden it can be...,sexist


In [13]:
#Rename
df_sexh = df_sexh.rename(columns={'label_sexist':'emotion'})
rows, columns = df_sexh.shape
print(f'Filas: {rows}\nColumnas: {columns}')
print(df_sexh['emotion'].unique())

Filas: 4854
Columnas: 2
['sexist']


In [14]:
# Concatenar todos los dataSets en uno solo
#La categoría political se eliminará, son muy ambiguos
df = pd.concat([ df_discg, df_racism,df_sexh], ignore_index=True)
rows, columns = df.shape
df = df.dropna()
df['emotion'] = df['emotion'].replace({'sexual':'Sexist','Saxual':'Sexist','sexist':'Sexist','Sexual ':'Sexist','Sexual':'Sexist',
                                       'Ethnicity':'Xenophobic','Ethnicity ':'Xenophobic','Religion':'Xenophobic','Religion ':'Xenophobic','Ethnically ':'Xenophobic',
                                       'racist':'Racist','Racism':'Racist',
                                       'Threats ':'Threats','Threat':'Threats','Threat ':'Threats','Religon':'Xenophobic','Religious ':'Xenophobic',
                                       'Religious':'Xenophobic','Religion':'Xenophobic','political':'Political','Political ':'Political',
                                       'Vocation':'Vocational','Vocational ':'Vocational','political ':'Political','Troll ':'Troll'})
df['emotion'] = df['emotion'].replace({'Vocational':'Troll','Racist':'Xenophobic'})
#Categorías a contar, Neutral, Xenophobic, Sexist, Threats Troll Same Vocational

df = df[df['emotion'] != 'Political']
df = df[df['emotion'] != 'Threats']
df = df[df['emotion'] != 'Troll']

n_cat = df['emotion'].value_counts()

n_target = df['emotion'].unique()
print(f'Valores target: {n_target}')
print(n_cat)

Valores target: ['Neutral' 'Xenophobic' 'Sexist']
emotion
Sexist        5882
Neutral       3365
Xenophobic    1950
Name: count, dtype: int64


In [15]:
df

,text,emotion
0,Ten outside soon doctor shake everyone treatme...,Neutral
1,my life has come to a standstill and at this p...,Neutral
2,girl this nigga make me sick to my stomach,Xenophobic
3,I wanna fuck you,Sexist
4,Haha isn't route running part of the position?...,Xenophobic
...,...,...
14021,The drama queen is just another open borders c...,Sexist
14022,This is easily the dumbest thing ever written....,Sexist
14023,Recently some thots started advertizing their ...,Sexist
14024,I wish yall understand what a burden it can be...,Sexist


In [16]:
#Limpieza de los valores
#Sinónimos sexual-Sexual,Saxual
#ethicall... Religion


In [17]:
#Aplicamos técnicas de PLN al df

stemmer = PorterStemmer()
encoder = LabelEncoder()

def tokenize_steam(sentence):
    words = nltk.word_tokenize(sentence)
    words = [stemmer.stem(w.lower()) for w in words if w.isalnum()]
    return words


In [18]:
#Diccionarios de reglas?
#Arquitectura, SVM, RedNeuronal?    
#Separamos los textos y las emociones 
all_words = []
df['text'].apply(lambda x: all_words.extend(tokenize_steam(x)))  # Agregamos palabra por palabra
all_words = sorted(set(all_words))

x = df['text']
y = df['emotion']

print(y.unique())

['Neutral' 'Xenophobic' 'Sexist']


In [20]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

#Partimos datos en prueba y Testing

#Vectorización
#v = CountVectorizer(vocabulary=all_words)
v = TfidfVectorizer(vocabulary=all_words)
#X_t = v.transform(x).toarray()
X_t = v.fit_transform(x).toarray()
y_t = encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, random_state=42, test_size=0.2)

model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu', kernel_regularizer=l2(0.003)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.002)))
model.add(Dropout(0.2))
model.add(Dense(len(encoder.classes_), activation='softmax'))

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1,validation_data=(X_test, y_test), callbacks=[early_stop])


/home/jordi/entornos/entorno_sic/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.6438 - loss: 0.9141 - val_accuracy: 0.8598 - val_loss: 0.5706
Epoch 2/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8722 - loss: 0.5352 - val_accuracy: 0.8844 - val_loss: 0.5279
Epoch 3/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8922 - loss: 0.4964 - val_accuracy: 0.8879 - val_loss: 0.5210
Epoch 4/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9037 - loss: 0.4768 - val_accuracy: 0.8799 - val_loss: 0.5210
Epoch 5/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9096 - loss: 0.4521 - val_accuracy: 0.8915 - val_loss: 0.5051
Epoch 6/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9104 - loss: 0.4480 - val_accuracy: 0.8973 - val_loss: 0.4933
Epoch 7/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9135 - loss: 0.4323 - val_accuracy: 0.8821 - val_loss: 0.4996
Epoch 8/100
560/560 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9203 - loss: 0.4226 - 

In [ ]:
import joblib
#detenerlo en el epoch 10 11?
#45 más viables
joblib.dump(model, '../models/RNA5.pkl')

['../models/RNA5.pkl']